In [5]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
from sklearn import preprocessing
import lightgbm as lgb
import os
import shutil
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from tqdm import tqdm
init_notebook_mode(connected=True)

pd.options.display.max_columns = 500
#import pandas_profiling

def display_importances(feature_importance_df_):
    cols = feature_importance_df_[["feature", "importance"]].groupby("feature").mean().sort_values(by="importance", ascending=False)[:50].index
    best_features = feature_importance_df_.loc[feature_importance_df_.feature.isin(cols)]
    plt.figure(figsize=(8, 10))
    sns.barplot(x="importance", y="feature", data=best_features.sort_values(by="importance", ascending=False))
    plt.title('LightGBM Features (avg over folds)')
    plt.tight_layout()
    #plt.savefig('lgbm_importances01.png')

In [6]:
train_df = pd.read_csv('../data/train.csv')
test_df = pd.read_csv('../data/test.csv')

var_list = [f for f in train_df.columns if 'var' in f]

In [7]:
freq_cols = [f +'_freq' for f in var_list]

test_df[freq_cols] = test_df[var_list].apply(lambda x: x.map(x.value_counts()))
train_df[freq_cols] = train_df[var_list].apply(lambda x: x.map(x.value_counts()))


test_df['min_freq'] = test_df[freq_cols].min(1)
train_df['min_freq'] = train_df[freq_cols].min(1)



real_test = test_df.loc[test_df.min_freq==1].copy()
fake_test = test_df.loc[test_df.min_freq!=1].copy()

In [8]:
from sklearn.model_selection import train_test_split
train_df_1,train_df_2 = train_test_split(train_df, test_size=0.5, random_state=42)


train_df_1[freq_cols] = train_df_1[var_list].apply(lambda x: x.map(x.value_counts()))
train_df_2[freq_cols] = train_df_2[var_list].apply(lambda x: x.map(x.value_counts()))
real_test[freq_cols] = real_test[var_list].apply(lambda x: x.map(x.value_counts()))
fake_test[freq_cols] = fake_test[var_list].apply(lambda x: x.map(x.value_counts()))

In [5]:
train_df = pd.concat([train_df_1, train_df_2],sort=False)
train_df.sort_index(inplace=True)

test_df = pd.concat([real_test, fake_test],sort=False)
test_df.sort_index(inplace=True)

In [6]:
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score

n_folds = 5
random_seed = 26
model = 'z_score_sample'


model_name = "{0}_{1}_folds".format(model, n_folds)
print("Model: {}".format(model_name))

Model: z_score_sample_5_folds


In [30]:
exclusion = ['ID_code', 'target'] + freq_cols
exclusion = ['ID_code', 'target'] 

# for var in tqdm(var_list):
#     exclusion.append('frequency_{}'.format(var))
#     exclusion.append('prob_{}'.format(var))
#     exclusion.append('true_prob_{}'.format(var))
    
feats = [c for c in train_df.columns if c not in exclusion]

import math 

In [31]:
feats

['var_0',
 'var_1',
 'var_2',
 'var_3',
 'var_4',
 'var_5',
 'var_6',
 'var_7',
 'var_8',
 'var_9',
 'var_10',
 'var_11',
 'var_12',
 'var_13',
 'var_14',
 'var_15',
 'var_16',
 'var_17',
 'var_18',
 'var_19',
 'var_20',
 'var_21',
 'var_22',
 'var_23',
 'var_24',
 'var_25',
 'var_26',
 'var_27',
 'var_28',
 'var_29',
 'var_30',
 'var_31',
 'var_32',
 'var_33',
 'var_34',
 'var_35',
 'var_36',
 'var_37',
 'var_38',
 'var_39',
 'var_40',
 'var_41',
 'var_42',
 'var_43',
 'var_44',
 'var_45',
 'var_46',
 'var_47',
 'var_48',
 'var_49',
 'var_50',
 'var_51',
 'var_52',
 'var_53',
 'var_54',
 'var_55',
 'var_56',
 'var_57',
 'var_58',
 'var_59',
 'var_60',
 'var_61',
 'var_62',
 'var_63',
 'var_64',
 'var_65',
 'var_66',
 'var_67',
 'var_68',
 'var_69',
 'var_70',
 'var_71',
 'var_72',
 'var_73',
 'var_74',
 'var_75',
 'var_76',
 'var_77',
 'var_78',
 'var_79',
 'var_80',
 'var_81',
 'var_82',
 'var_83',
 'var_84',
 'var_85',
 'var_86',
 'var_87',
 'var_88',
 'var_89',
 'var_90',
 'var_91'

In [32]:
clfs = []
folds = StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=random_seed)
oof_preds = np.zeros((len(train_df), 1))
test_preds = np.zeros((len(test_df), 1))


X = train_df[feats]
y = train_df['target']
X_test = test_df[feats]
test_ids = test_df.ID_code.values
X['target'] = train_df['target']

parameters = {
    'objective': 'binary',
    'metric': 'auc',
    'is_unbalance': 'true',
    #'scale_pos_weight': 400,
    #'device' : 'gpu' ,
    'boosting': 'gbdt',
    'num_leaves': 5, #31
    'feature_fraction': 0.3,
    'bagging_fraction': 0.7,
    'bagging_freq': 10,
    'learning_rate': 0.05, #0.05
    'verbose': 30
    #'min_data_in_leaf': 200
}

feature_importance_df = pd.DataFrame()
for fold_, (trn_, val_) in enumerate(folds.split(X, y)):
    print("Current Fold: {}".format(fold_+1))
    trn_x, trn_y = X.iloc[trn_, :], y[trn_]
    val_x, val_y = X.iloc[val_, :], y[val_]
    
#     X_test[freq_cols] = X_test[var_list].apply(lambda x: x.map(x.value_counts()))
#     trn_x[freq_cols] = trn_x[var_list].apply(lambda x: x.map(x.value_counts()))
#     val_x[freq_cols] = val_x[var_list].apply(lambda x: x.map(x.value_counts()))
    
    z_pos_cols = []
    z_neg_cols = []
    z_diff_cols = []
    for var in var_list:
        
        #pos
        trn_x[f'z_{var}_pos'] = (trn_x[var] - trn_x.loc[trn_x.target==1, var].mean())/(trn_x.loc[trn_x.target==1, var].std()/(trn_x[f'{var}_freq'].apply(lambda x: math.sqrt(x))))
        val_x[f'z_{var}_pos'] = (val_x[var] - trn_x.loc[trn_x.target==1, var].mean())/(trn_x.loc[trn_x.target==1, var].std()/(val_x[f'{var}_freq'].apply(lambda x: math.sqrt(x))))
        X_test[f'z_{var}_pos'] = (X_test[var] - trn_x.loc[trn_x.target==1, var].mean())/(trn_x.loc[trn_x.target==1, var].std()/(X_test[f'{var}_freq'].apply(lambda x: math.sqrt(x))))

        
        trn_x[f'z_{var}_neg'] = (trn_x[var] - trn_x.loc[trn_x.target==0, var].mean())/(trn_x.loc[trn_x.target==0, var].std()/(trn_x[f'{var}_freq'].apply(lambda x: math.sqrt(x))))
        val_x[f'z_{var}_neg'] = (val_x[var] - trn_x.loc[trn_x.target==0, var].mean())/(trn_x.loc[trn_x.target==0, var].std()/(val_x[f'{var}_freq'].apply(lambda x: math.sqrt(x))))
        X_test[f'z_{var}_neg'] = (X_test[var] - trn_x.loc[trn_x.target==0, var].mean())/(trn_x.loc[trn_x.target==0, var].std()/(X_test[f'{var}_freq'].apply(lambda x: math.sqrt(x))))

        trn_x[f'z_{var}_diff'] = trn_x[f'z_{var}_pos'] - trn_x[f'z_{var}_neg']
        val_x[f'z_{var}_diff'] = val_x[f'z_{var}_pos'] - val_x[f'z_{var}_neg']
        X_test[f'z_{var}_diff'] = X_test[f'z_{var}_pos'] - X_test[f'z_{var}_neg']
        
        z_pos_cols.append(f'z_{var}_pos')
        z_neg_cols.append(f'z_{var}_neg')
        z_diff_cols.append(f'z_{var}_diff')
        
#     X_test.drop(z_pos_cols,axis=1,inplace=True)
#     val_x.drop(z_pos_cols,axis=1,inplace=True)
#     trn_x.drop(z_pos_cols,axis=1,inplace=True)
        
#     X_test.drop(z_neg_cols,axis=1,inplace=True)
#     val_x.drop(z_neg_cols,axis=1,inplace=True)
#     trn_x.drop(z_neg_cols,axis=1,inplace=True)
        
#     X_test.drop(freq_cols,axis=1,inplace=True)
#     val_x.drop(freq_cols,axis=1,inplace=True)
#     trn_x.drop(freq_cols,axis=1,inplace=True)
    
#     #X_test.drop(freq_cols,axis=1,inplace=True)
#     val_x.drop('target',axis=1,inplace=True)
#     trn_x.drop('target',axis=1,inplace=True)   

    exclusion = ['ID_code', 'target'] + freq_cols + z_diff_cols

    # for var in tqdm(var_list):
    #     exclusion.append('frequency_{}'.format(var))
    #     exclusion.append('prob_{}'.format(var))
    #     exclusion.append('true_prob_{}'.format(var))

    feats = [c for c in trn_x.columns if c not in exclusion]



    trn_lgb = lgb.Dataset(trn_x[feats], trn_y)
    val_lgb = lgb.Dataset(val_x[feats], val_y)
    clf = lgb.train(parameters,
                     train_set=trn_lgb,
                     #valid_sets=[valid_data_lgb,holdout_data_lgb],
                     valid_sets=[trn_lgb, val_lgb],
                     num_boost_round=30000,
                     early_stopping_rounds=50,
                     verbose_eval=500)
    


    val_pred = clf.predict(val_x[feats])
    test_fold_pred = clf.predict(X_test[feats])

    print("AUC = {}".format(roc_auc_score(val_y, val_pred)))
    oof_preds[val_, :] = val_pred.reshape((-1, 1))
    test_preds += test_fold_pred.reshape((-1, 1))
    
   # print('getting feature importance')
    
#     fold_importance_df = pd.DataFrame()
#     fold_importance_df["feature"] = feats
#     fold_importance_df["importance"] = clf.feature_importance()
#     fold_importance_df["fold"] = fold_ + 1
#     feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)
    
    
test_preds /= n_folds
roc_score = roc_auc_score(y, oof_preds.ravel())
print("Overall AUC = {}".format(roc_score))


print("Saving submission file")
sample = pd.read_csv('../data/sample_submission.csv')
sample.target = test_preds.astype(float)
sample.ID_code = test_ids
sample.to_csv('../submissions/{}_{}.csv'.format(model_name,str(roc_score)), index=False)

#display_importances(feature_importance_df)


Current Fold: 1
Training until validation scores don't improve for 50 rounds.
[500]	training's auc: 0.895166	valid_1's auc: 0.871641
[1000]	training's auc: 0.916696	valid_1's auc: 0.889555
[1500]	training's auc: 0.926578	valid_1's auc: 0.896222
[2000]	training's auc: 0.933381	valid_1's auc: 0.898394
Early stopping, best iteration is:
[2135]	training's auc: 0.935071	valid_1's auc: 0.898923
AUC = 0.8989234782702479
Current Fold: 2
Training until validation scores don't improve for 50 rounds.
[500]	training's auc: 0.895444	valid_1's auc: 0.871529
[1000]	training's auc: 0.916737	valid_1's auc: 0.888198
[1500]	training's auc: 0.926767	valid_1's auc: 0.894011
[2000]	training's auc: 0.933351	valid_1's auc: 0.896417
Early stopping, best iteration is:
[2063]	training's auc: 0.934147	valid_1's auc: 0.896551
AUC = 0.8965513031269542
Current Fold: 3
Training until validation scores don't improve for 50 rounds.
[500]	training's auc: 0.8953	valid_1's auc: 0.87776
[1000]	training's auc: 0.916316	vali